# 11.02 - Introducción a Polars para Usuarios de Pandas

**Autor:** Miguel Angel Vazquez Varela  
**Nivel:** Avanzado  
**Tiempo estimado:** 40 min

---

## ¿Qué aprenderemos?

- Por qué usar Polars (velocidad y multi-threading)
- Diferencias clave en la API frente a Pandas
- El concepto de Lazy Evaluation
- Operaciones básicas: Select, Filter, GroupBy y Join

In [1]:
import pandas as pd
import numpy as np
import time

try:
    import polars as pl
    POLARS_AVAILABLE = True
    print(f"Polars version: {pl.__version__}")
except ImportError:
    POLARS_AVAILABLE = False
    print("Polars no instalado. Instalar con: pip install polars")

Polars version: 1.29.0


---

## 1. Rendimiento: Polars vs Pandas

Crearemos un dataset de 1 millón de filas para comparar la velocidad de agregación.

In [2]:
size = 1_000_000
df_pd = pd.DataFrame({
    'group': np.random.choice(['A', 'B', 'C', 'D'], size=size),
    'value': np.random.randn(size)
})

if POLARS_AVAILABLE:
    df_pl = pl.from_pandas(df_pd)

# Tiempo Pandas
start = time.time()
res_pd = df_pd.groupby('group')['value'].mean()
print(f"Tiempo Pandas: {time.time() - start:.4f}s")

if POLARS_AVAILABLE:
    # Tiempo Polars
    start = time.time()
    res_pl = df_pl.group_by('group').agg(pl.col('value').mean())
    print(f"Tiempo Polars: {time.time() - start:.4f}s")

Tiempo Pandas: 0.0244s
Tiempo Polars: 0.0567s


---

## 2. API Expresiva

Polars elimina el concepto de `index` y utiliza expresiones que se pueden encadenar.

In [3]:
if POLARS_AVAILABLE:
    # Encadenamiento de expresiones (Select + Filter)
    query = (
        df_pl
        .filter(pl.col('value') > 0)
        .select([
            pl.col('group'),
            (pl.col('value') * 100).alias('scaled_value')
        ])
        .head(5)
    )
    print(query)

shape: (5, 2)
┌───────┬──────────────┐
│ group ┆ scaled_value │
│ ---   ┆ ---          │
│ str   ┆ f64          │
╞═══════╪══════════════╡
│ C     ┆ 100.782788   │
│ D     ┆ 40.315562    │
│ B     ┆ 54.585096    │
│ D     ┆ 72.867927    │
│ D     ┆ 31.487054    │
└───────┴──────────────┘


---

## 3. Lazy Evaluation

Esta es la característica más potente de Polars: no ejecuta la consulta hasta que se lo pides, permitiendo al optimizador de planos de consulta hacer su trabajo.

In [4]:
if POLARS_AVAILABLE:
    lazy_plan = (
        df_pl.lazy()
        .filter(pl.col('group') == 'A')
        .group_by('group')
        .agg(pl.col('value').sum())
    )
    
    print("Plan de consulta (sin ejecutar):")
    print(lazy_plan.explain())
    
    print("\nResultado ejecutado con .collect():")
    print(lazy_plan.collect())

Plan de consulta (sin ejecutar):
AGGREGATE
  [col("value").sum()] BY [col("group")]
  FROM
  FILTER [(col("group")) == ("A")]
  FROM
    DF ["group", "value"]; PROJECT["value", "group"] 2/2 COLUMNS

Resultado ejecutado con .collect():
shape: (1, 2)
┌───────┬────────────┐
│ group ┆ value      │
│ ---   ┆ ---        │
│ str   ┆ f64        │
╞═══════╪════════════╡
│ A     ┆ -96.537543 │
└───────┴────────────┘


---

## Resumen

- **Polars** es mucho más rápido para grandes volúmenes de datos gracias a Rust y Apache Arrow.
- **No tiene Index**: Todo se maneja a través de columnas.
- **Lazy API**: Usa `.lazy()` y `.collect()` para maximizar el rendimiento.